In [1]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import platform
import sys
import sklearn

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from matplotlib import pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report

from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn import  metrics
from time import time
import scipy.stats as st
from operator import itemgetter
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import numpy


import xgboost as xgb



In [2]:

print('Operating system version....', platform.platform())
print("Python version is........... %s.%s.%s" % sys.version_info[:3])
print('scikit-learn version is.....', sklearn.__version__)
print('pandas version is...........', pd.__version__)
print('numpy version is............', np.__version__)
print('matplotlib version is.......', matplotlib.__version__)
print('scipy version is.......', scipy.__version__)

Operating system version.... Windows-10-10.0.16299-SP0
Python version is........... 3.6.3
scikit-learn version is..... 0.19.1
pandas version is........... 0.22.0
numpy version is............ 1.12.1
matplotlib version is....... 2.1.2
scipy version is....... 1.0.0


In [26]:
class Timer:
  def __init__(self):
    self.start = time.time()

  def restart(self):
    self.start = time.time()

  def get_time(self):
    end = time.time()
    m, s = divmod(end - self.start, 60)
    h, m = divmod(m, 60)
    time_str = "%02d:%02d:%02d" % (h, m, s)
    return time_str



In [27]:

def split_dataset(dataset, train_percentage, feature_headers,
                                  target_header,random_state=42,mode_debug=0):

    train_x, test_x, train_y, test_y = train_test_split(dataset[feature_headers], dataset[target_header],
                                                        train_size=train_percentage, test_size=None, random_state=42)


    if mode_debug==1:
        # Train and Test dataset size details
        print("--------------------------------")
        print("Train_x Shape :: ", train_x.shape)
        print("Train_y Shape :: ", train_y.shape)
        print("Test_x Shape :: ", test_x.shape)
        print("Test_y Shape :: ", test_y.shape)
        print("--------------------------------")


In [28]:
def lecture_data(Fichier, xnames, xindex_col, allure=1, mode_debug=0,avec_index=True):


    

    if  avec_index==True:
        df = pd.read_csv(Fichier,
                     index_col=xindex_col,
                     sep=';',
                     names=xnames, skipinitialspace=True,
                     encoding='Latin-1')
    else:
        df = pd.read_csv(Fichier,
                         index_col=None,
                         sep=';',
                         names=xnames, skipinitialspace=True,
                         encoding='Latin-1')



    print("\nDF Après LECTURE -------------- ",Fichier)

    print("\n",df.shape)
#    print("\n",df.head(5))


    if  avec_index==False:
        df.drop(["IDPARTCIPANT"], axis=1, inplace=True)
        df.drop(["IDCOURSE"], axis=1, inplace=True)


    df.drop(["FIN_ligne"], axis=1, inplace=True)
    df.drop(["PAR_NP"], axis=1, inplace=True)
    df.drop(["cendre"], axis=1, inplace=True)
    df.drop(["CO_PRIX"], axis=1, inplace=True)
    df.drop(["PAR_CLASSE_AGE"], axis=1, inplace=True)
    #df.drop(["PAR_AGE"], axis=1, inplace=True)
    df.drop(["grande_piste"], axis=1, inplace=True)
    df.drop(["PAR_COTEDER"], axis=1, inplace=True)
    #df=my_drop(df, "HIPPO")
    df=my_drop(df, "PAR_AGE")
    df = my_drop(df, "PAR_GAIN")



    if allure==1:
        df.drop(["POIDS"], axis=1, inplace=True)


    if allure == 2:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)

    if allure == 3:
        df.drop(["POIDS"], axis=1, inplace=True)

    if allure == 4:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)

    if allure == 5:
        df.drop(["autostart"], axis=1, inplace=True)
        df.drop(["CO_DISTANCE"], axis=1, inplace=True)





    return df


def my_split(taille_entrainement, df,feature_columns,response_column ,random_state=42,mode_debug=0):
    train_x, test_x, train_y, test_y = split_dataset(df,
                                                         taille_entrainement,
                                                          feature_columns,
                                                          response_column,
                                                         random_state=random_state,
                                                     mode_debug=mode_debug
                                                     )

    return train_x, test_x, train_y, test_y


In [29]:
def my_drop(df, col):

    if col in df:
        df.drop([col], axis=1, inplace=True)
    return df


# Creating bins for the win column
def assign_selection(W):
    if W <5:
        return 1
    else:
        return 0

    
def save_mymodel(clf,allure):
    joblib.dump(clf,'diabolo'+str(allure)+'.pkl')


def load_mymodel(allure):
    clf=joblib.load('diabolo'+str(allure)+'.pkl')
    return clf


#Définir une méthode pour tracer l'importance du prédicteur
def Plot_predictor_importance(best_model, feature_columns):
    feature_importance = best_model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    y_pos  = np.arange(sorted_idx.shape[0]) + .7
    fig, ax = plt.subplots( figsize=(10, 10))



    ax.barh(y_pos,
            feature_importance[sorted_idx],
            align='center',
            color='green',
            ecolor='black',
            height=0.8)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_columns)
    ax.invert_yaxis()



    ax.set_xlabel('Relative Importance')
    ax.set_title('Predictor Importance')
    plt.show()
    
    
def calul_data_allure(allure_etudier, mode_debug=0,avec_index=True):

    xnames = ['ALLURE'  , 'CO_DISTANCE',
              'CO_PRIX',  'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE',      'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant','PAR_PROPRIO',
                                         'NOM_JOC',
                                         'NOM_ENTR',
                                        'POIDS',
                                         'FIN_ligne']








    df = lecture_data('d:\diabollo_al_1.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure_etudier,avec_index=avec_index)


    # groupage par allure
    df = df.groupby("ALLURE")
    df = df.get_group(allure_etudier)
    #df.head(5)
    df = my_drop(df, 'ALLURE')
    # Contruction de df_gagant
    df_gagnant = df
    # les lignes sans arrivée n'apportent rien !!!!
    df_gagnant = df_gagnant[df_gagnant.PAR_ARRIVE > 0]  # On garde la ligne qui possede information arrivée
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN < 20000000]
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN > 0]



    if allure_etudier==2:
        df_gagnant = df_gagnant[df_gagnant.POIDS > 0]
        df_gagnant = df_gagnant[df_gagnant.POIDS < 100]



    # Shape des données df_gagant
    if  mode_debug==1:
        print(" df_gagnant ", df_gagnant.shape)
        COLUM = df_gagnant.columns
        print("Features", COLUM )

    # Construction de la cible
    df_gagnant['SELECTION'] = df_gagnant['PAR_ARRIVE'].apply(assign_selection)

    #df_gagnant = df_gagnant[df_gagnant.PAR_COTEDER < 150]
     # Suppression de la colonne qui a permis de construire la cible : PAR_ARRIVE
    df_gagnant.drop(["PAR_ARRIVE"], axis=1, inplace=True)

    print("\ndf_gagnant \n", df_gagnant.shape)  # description de l'ensemble
        # pd.DataFrame.hist(df_gagnant, figsize = [15,15]);


    df_gagnant_len            = len(df_gagnant.columns) - 1
    Lib_features                = list(df_gagnant.columns[:df_gagnant_len])  # Liebllé des variable DATA
    feature_columns         = Lib_features ##<<<<<<<<<<<<<<<<
    n_features = len(feature_columns) ##<<<<<<<<<<<<<<<<
    response_column = ['SELECTION']  ##<<<<<<<<<<<<<<<<



    print("Features                   :  ", Lib_features, " <<<<************")

    print("Nombre de feature   :  ", n_features)
    print("Colonne cible            : ", response_column)
    print("FEATURES COLONNES            : ", feature_columns)
    print("\n\n------------------------------------------------")


    # ---------------------------------------------------
    return df_gagnant, n_features, feature_columns, response_column    
    
    

In [30]:
#****************************************************************************************************************
def entraine_allure(allure, nb_iter,mode_debug=0, actualise=0):
    # LECTURE DATA
    print('---------------------ALLURE(  %s  )------------------------------------------------' % allure)
    # Le fichier est lu meme si on  a le best model
    df_gagnant, n_features, feature_columns, response_column = lecture_data.calul_data_allure(allure_etudier = allure,mode_debug=mode_debug)


    if actualise==0:

        #estimateur = GradientBoostingClassifier
        estimateur = XGBClassifier
        taille_training = 0.85
        train_x, test_x, train_y, test_y = my_split(df=df_gagnant,
                                                                              taille_entrainement=taille_training,
                                                                              feature_columns =feature_columns,
                                                                              response_column =response_column,
                                                                               random_state=42,
                                                                               mode_debug=mode_debug)
        #best_model= analyse.fit_xg(X_train=train_x, y_train=train_y, X_test =test_x,y_test=test_y)
        best_model = my_fit(estimateur=estimateur,
                                        nb_iter=nb_iter, my_nb_splits=10,
                                       my_test_size=1 - taille_training,
                                        my_random_state=0,
                                        train_x=train_x, train_y=train_y,allure=allure,featurecolums=feature_columns)



        Plot_predictor_importance(best_model, feature_columns)



        # sauvegarde du modele
        save_mymodel(clf=best_model,allure=allure)

        # EVALUATION
        evaluation(mybest_model=best_model, train_x=train_x, test_x=test_x,
                           train_y=train_y, test_y=test_y, mode_debug=mode_debug)

    else:
        # CHARGEMENT DU MODELE
        best_model = lecture_data.load_mymodel(allure)




    return best_model, feature_columns, response_column



#Définir une fonction d'utilité pour signaler les meilleurs scores
def Report_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

            

In [31]:


def afficheEvalSet(mybest_model,
                    set_train,
                   set_train_cible,
                   set_test,
                   set_test_cible,
                   mode_debug=0,
                   type_eval=1):


    set_test_cible_predicted = mybest_model.predict(set_test)
    set_train_cible_predicted = mybest_model.predict(set_train)

    if type_eval==1:
                print("Evaluation TEST-----------------------------------------------------------------------")

                set_test_cible_probabilities = mybest_model.predict_proba(set_test)
                auc = roc_auc_score(set_test_cible, set_test_cible_predicted)



    else:
                print("Evaluation TRAINING--------------------------------------------------------------------")

                set_train_cible_probabilities = mybest_model.predict_proba(set_train)
                auc = roc_auc_score(set_train_cible, set_train_cible_predicted)



    print("\n roc_auc_score :  ", auc)
    if mode_debug == 1:
        for x, y in [(set_train, set_train_cible), (set_test, set_test_cible)]:
            yp = mybest_model.predict(x)
            cm = confusion_matrix(y, yp.ravel())
            print(cm)

    import matplotlib.pyplot as plt

    if mode_debug == 1:
        plt.matshow(cm)
        plt.title('Confusion matrix')
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

    if type_eval==1:

        ntotal = len(set_test)
        correct = set_test_cible["SELECTION"].ravel() == set_test_cible_predicted
        numCorrect = sum(correct)
        percent = round((100.0 * numCorrect) / ntotal, 6)

        print("Classification Correcte des données de test : {0:d}/{1:d}  {2:8.3f}%".format(numCorrect, ntotal, percent))
        prediction_score = 100.0 * mybest_model.score(set_test, set_test_cible)
        print('\nScore  TEST  : %8.3f  ************************' % prediction_score)

    else:

        ntotal = len(set_train)
        correct = set_train_cible["SELECTION"].ravel() == set_train_cible_predicted
        numCorrect = sum(correct)
        percent = round((100.0 * numCorrect) / ntotal, 6)

        print("\n CLASSIFICATION CORRECTE DES DONNEES DE Train : {0:d}/{1:d}  {2:8.3f}%".format(numCorrect, ntotal, percent))
        prediction_score = 100.0 * mybest_model.score(set_train, set_train_cible)
        print('\nScore  TRAINING  : %8.3f  ************************' % prediction_score)

        
def evaluation(mybest_model,
                  train_x, test_x,
                   train_y, test_y,
                    mode_debug=0):


    afficheEvalSet(mybest_model=mybest_model,
                         set_train=train_x,
                         set_train_cible=train_y,
                         set_test=test_x,
                          set_test_cible=test_y,
                   mode_debug=mode_debug, type_eval=1)

    afficheEvalSet(mybest_model=mybest_model,
                         set_train=train_x,
                         set_train_cible=train_y,
                         set_test=test_x,
                          set_test_cible=test_y,
                   mode_debug=mode_debug, type_eval=2)



In [32]:

def  my_randomize_estimateur(estimateur, train_x, train_y,nb_iteration, verbose,allure,my_scoring="roc_auc"):



        #param_grid = dict(learning_rate=learning_rate,                          n_estimators=n_estimators)

        if allure==1:
            param_grid = {
             'silent': [False],
             'max_depth': [6 ],
              #'min_samples_split': [3000,3200],
             'learning_rate': [0.05],
             'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
             'gamma': [0.25],
            'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
            'n_estimators': [100]
           }


        if allure==2:
              param_grid = {        'silent': [False],
                   'max_depth': [7,14] ,
                    'learning_rate': [0.05],
                   'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                   'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                   'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                   'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
                   'gamma': [0.25],
                   'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
                  'n_estimators': [150,175,200]
              }

        if allure == 3:
            param_grid = {'silent': [False],
                          'max_depth': [7],
                          #  'min_samples_split': [2500, 3000, 4000],
                          'learning_rate': [0.05],
                          'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
                          'gamma': [0.25],
                          'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
                          'n_estimators': [150]
                          }
        if allure == 4:
            param_grid = {'silent': [False],
                          'max_depth': [7],
                          #  'min_samples_split': [2500, 3000, 4000],
                          'learning_rate': [0.05],
                          'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                          'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
                          'gamma': [0.25],
                          'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
                          'n_estimators': [150]
                          }
        if allure == 5:
                param_grid = {'silent': [False],
                              'max_depth': [7],
                              #  'min_samples_split': [2500, 3000, 4000],
                              'learning_rate': [0.05],
                              'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                              'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                              'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                              'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
                              'gamma': [0.25],
                              'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
                              'n_estimators': [150]
                              }

        clf=XGBClassifier()
        estimator = RandomizedSearchCV(clf,
                                    param_distributions=param_grid, scoring=my_scoring,
                                     verbose=3, n_iter=nb_iteration,n_jobs=-1,cv=4)


In [33]:


def my_fit(estimateur,          nb_iter, my_nb_splits, my_test_size,
                                    my_random_state, train_x, train_y, featurecolums, allure, mode_debug=0):

    from sklearn.model_selection import cross_val_score



    my_timer = Timer()

    estimator = my_randomize_estimateur(estimateur, train_x=train_x,train_y=train_y, allure=allure,nb_iteration=nb_iter, verbose=2, my_scoring="roc_auc")




    estimator.fit(train_x, train_y["SELECTION"].ravel(), verbose=True)


    print("Grid scores on development set:")
    print()
    means = estimator.cv_results_['mean_test_score']
    print(means)

    stds = estimator.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, estimator.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    elapsed = my_timer.get_time()
    print("\n\nTemps de calcul du FIT  est : %s" % elapsed)

    print("Report score \n")
    Report_scores(estimator.cv_results_, n_top=nb_iter)
    # Plot_learning_curve(estimator, '', train_x, train_y["SELECTION"].ravel(),cv=cv_, n_jobs=4)
    best_model = estimator.best_estimator_  # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    print('\nbest_model                                  : ', best_model)
    # print('\nbest_model Nombre de Feature  :  ', best_model.n_features_)
    print('\nbest_model  Classes                      : ', best_model.classes_)
   # print('\nbest_model  best_params_           : ', best_model.best_params_)

    best_score = estimator.best_score_  # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    print("\nBEST SCORE = ", best_score)

    return best_model

In [34]:

def calul_data_allure(allure_etudier, mode_debug=0,avec_index=True):

    xnames = ['ALLURE'  , 'CO_DISTANCE',
              'CO_PRIX',  'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE',      'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant','PAR_PROPRIO',
                                         'NOM_JOC',
                                         'NOM_ENTR',
                                        'POIDS',
                                         'FIN_ligne']








    df = lecture_data('d:\diabollo_al_1.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure_etudier,avec_index=avec_index)


    # groupage par allure
    df = df.groupby("ALLURE")
    df = df.get_group(allure_etudier)
    #df.head(5)
    df = my_drop(df, 'ALLURE')
    # Contruction de df_gagant
    df_gagnant = df
    # les lignes sans arrivée n'apportent rien !!!!
    df_gagnant = df_gagnant[df_gagnant.PAR_ARRIVE > 0]  # On garde la ligne qui possede information arrivée
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN < 20000000]
    #df_gagnant = df_gagnant[df_gagnant.PAR_GAIN > 0]



    if allure_etudier==2:
        df_gagnant = df_gagnant[df_gagnant.POIDS > 0]
        df_gagnant = df_gagnant[df_gagnant.POIDS < 100]



    # Shape des données df_gagant
    if  mode_debug==1:
        print(" df_gagnant ", df_gagnant.shape)
        COLUM = df_gagnant.columns
        print("Features", COLUM )

    # Construction de la cible
    df_gagnant['SELECTION'] = df_gagnant['PAR_ARRIVE'].apply(assign_selection)

    #df_gagnant = df_gagnant[df_gagnant.PAR_COTEDER < 150]
     # Suppression de la colonne qui a permis de construire la cible : PAR_ARRIVE
    df_gagnant.drop(["PAR_ARRIVE"], axis=1, inplace=True)

    print("\ndf_gagnant \n", df_gagnant.shape)  # description de l'ensemble
        # pd.DataFrame.hist(df_gagnant, figsize = [15,15]);


    df_gagnant_len            = len(df_gagnant.columns) - 1
    Lib_features                = list(df_gagnant.columns[:df_gagnant_len])  # Liebllé des variable DATA
    feature_columns         = Lib_features ##<<<<<<<<<<<<<<<<
    n_features = len(feature_columns) ##<<<<<<<<<<<<<<<<
    response_column = ['SELECTION']  ##<<<<<<<<<<<<<<<<



    print("Features                   :  ", Lib_features, " <<<<************")

    print("Nombre de feature   :  ", n_features)
    print("Colonne cible            : ", response_column)
    print("FEATURES COLONNES            : ", feature_columns)
    print("\n\n------------------------------------------------")


    # ---------------------------------------------------
    return df_gagnant, n_features, feature_columns, response_column

In [35]:
def construireFichierCSV(allure, best_model, feature_columns, response_column, mode_debug=0):
    # lecture des données a jouer


    xnames = ['ALLURE', 'CO_DISTANCE',
              'CO_PRIX', 'HIPPO',
              'IDCOURSE', 'IDPARTCIPANT',
              'PAR_AGE', 'PAR_ARRIVE',
              'PAR_CARRIERE',
              'PAR_CARRIERE_Q',
              'PAR_CLASSE_AGE',
              'PAR_COTEDER',
              'PAR_ENT_ECART_GAGNANT',
              'PAR_ENT_RAPPORT_GAGNANT_M',
              'PAR_ENT_REU_PLACE',
              'PAR_ENT_REUSSITE_GAGNE',
              'PAR_ENT_VICTOIRE',
              'PAR_GAIN',
              'pAR_JOC_ECART_GAGNANT',
              'PAR_JOC_ECART_PLACE',
              'PAR_JOC_NB_COURSE',
              'PAR_JOC_PLACE_3P',
              'pAR_JOC_RAPPORT_GAGNANT_M',
              'PAR_JOC_REU_PLACE',
              'pAR_JOC_REUSSITE_GAGNE',
              'pAR_JOC_VICTOIRE',
              'PAR_NP',
              'PAR_NUM',
              'PAR_PLACE',
              'PAR_PLACE_Q',
              'PAR_REUSSITE_3P',
              'PAR_REUSSITE_GAGNE',
              'PAR_REUSSITE_QUINTE',
              'PAR_RUESSITE_PLACE',
              'autostart',
              'cendre',
              'grande_piste',
              'Point',
              'Nb_partant', 'PAR_PROPRIO',
              'NOM_JOC',
              'NOM_ENTR',
              'POIDS',
              'FIN_ligne']



    df_numero_a_predire = lecture_data.lecture_data('d:\diabollo_al_1_D.csv', xnames, ['IDPARTCIPANT', 'IDCOURSE'],allure=allure)



    if allure==5:
        print("\nSHAPE Numero a predire")
        print(df_numero_a_predire.shape)



    df_numero_a_predire = df_numero_a_predire.groupby("ALLURE")
    df_numero_a_predire = df_numero_a_predire.get_group(allure)
    df_numero_a_predire = lecture_data.my_drop(df_numero_a_predire, 'ALLURE')

    df_numero_a_predire = lecture_data.my_drop(df_numero_a_predire, 'PAR_ARRIVE')

    if mode_debug == 1:
        print("\nSHAPE Numero a predire")
        print(df_numero_a_predire.shape)

    my_timer = analyse.Timer()


    df_numero_a_predire['SELECTION'] = 0

    train_x, test_x, train_y, test_y = analyse.split_dataset(df_numero_a_predire, 0, feature_columns, response_column)
   # print(test_x.head(2))
    print(test_x.shape)


    # sauvegarde
    y_pred = best_model.predict(test_x)
    if mode_debug == 1:
        print("y pred")
        print(y_pred)

    df_pred = pd.DataFrame.from_dict(y_pred)
    test_copy = test_x.copy()


    # print(df_pred)
    #print("PREDICTION COURSES A JOUER")
    if mode_debug==1:
        print(y_pred)


    # UTILISATION DE BEST_MODEL
    PROBA = best_model.predict_proba(test_x)
    #print("PROBA \n %s" ,PROBA)

    df_proba = pd.DataFrame.from_dict(PROBA)

    #print("df final=")
    df_final = pd.concat([df_proba, df_pred], axis=1)

    if mode_debug==1:
        print("SHAPE df_final")
        print(df_final.shape)


    test_x = test_copy.copy()

    test_x = lecture_data.my_drop(test_x, 'PAR_CLASSE_AGE')

    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_ECART_GAGNANT')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_RAPPORT_GAGNANT_M')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_REU_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'PAR_ENT_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_GAIN')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_ECART_GAGNANT')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_ECART_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_NB_COURSE')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_PLACE_3P')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_QUINTE')
    test_x = lecture_data.my_drop(test_x, 'PAR_RUESSITE_PLACE')
    test_x = lecture_data.my_drop(test_x, 'CO_PRIX')
    test_x = lecture_data.my_drop(test_x, 'PAR_CARRIERE')
    test_x = lecture_data.my_drop(test_x, 'PAR_CARRIERE_Q')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_RAPPORT_GAGNANT_M')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_REUSSITE_GAGNE')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_PLACE_Q')
    test_x = lecture_data.my_drop(test_x, 'PAR_CLASSE_AGE')
    test_x = lecture_data.my_drop(test_x, 'pAR_JOC_VICTOIRE')
    test_x = lecture_data.my_drop(test_x, 'PAR_REUSSITE_3P')
    test_x = lecture_data.my_drop(test_x, 'PAR_JOC_REU_PLACE')
    test_x = lecture_data.my_drop(test_x, 'PAR_NUM')
    test_x = lecture_data.my_drop(test_x, 'PAR_COTEDER')
    test_x = lecture_data.my_drop(test_x, 'musique')
    test_x = lecture_data.my_drop(test_x, 'Nb_partant')

    test_x = lecture_data.my_drop(test_x, 'autostart')
    test_x = lecture_data.my_drop(test_x, 'grande_piste')
    test_x = lecture_data.my_drop(test_x, 'cendre')

    test_x = lecture_data.my_drop(test_x, 'PAR_PROPRIO')
    test_x = lecture_data.my_drop(test_x, 'NOM_JOC')
    test_x = lecture_data.my_drop(test_x, 'NOM_ENTR')

    test_x = lecture_data.my_drop(test_x, 'HIPPO')
    test_x = lecture_data.my_drop(test_x, 'PAR_AGE')
    test_x = lecture_data.my_drop(test_x, 'POIDS')
    test_x = lecture_data.my_drop(test_x, 'CO_DISTANCE')
    test_x = lecture_data.my_drop(test_x, 'CO_PRIX')

    #print("test_x",test_x.shape)




    #print(test_x)


    test_x['v0'] = 0.0
    test_x['v1'] = 0.0
    test_x['sel'] = 0
    #   print("test_x", test_x.shape)
    # test_x
    cumul = 1
    nb_rows = len(df_proba.index)
    print("nb_rows=", nb_rows)

    for i in range(0, nb_rows):
        n = df_proba[0][i]
        test_x['v0'][i] = n

        n = df_proba[1][i]
        test_x['v1'][i] = n


    #print("test_x",test_x.shape)

   # print("\n\nTEST_X\n")
   # print(test_x.head())

    if (allure == 1):
        test_x.to_csv("d:\py_resultat_trot.csv")

    if (allure == 2):
        test_x.to_csv("d:\py_resultat_galop.csv")

    if (allure == 3):
        test_x.to_csv("d:\py_resultat_trot_monte.csv")

    if (allure == 4):
        test_x.to_csv("d:\py_resultat_haie.csv")

    if (allure == 5):
        test_x.to_csv("d:\py_resultat_steeple.csv")

    elapsed = my_timer.get_time()

    print("\nTemps de calcul de la construction du fichier  est : %s" % elapsed)

    print("\n\n ALLER DANS APPLICATION DIABOLO ....")


In [36]:
def entraine_allure(allure, nb_iter,mode_debug=0, actualise=0):
    # URE DATA
    print('---------------------ALLURE(  %s  )------------------------------------------------' % allure)
    # Le fichier est lu meme si on  a le best model
    df_gagnant, n_features, feature_columns, response_column = calul_data_allure(allure_etudier = allure,mode_debug=mode_debug)


    if actualise==0:

        #estimateur = GradientBoostingClassifier
        estimateur = XGBClassifier
        taille_training = 0.85
        train_x, test_x, train_y, test_y = my_split(df=df_gagnant,
                                                                              taille_entrainement=taille_training,
                                                                              feature_columns =feature_columns,
                                                                              response_column =response_column,
                                                                               random_state=42,
                                                                               mode_debug=mode_debug)
        #best_model= analyse.fit_xg(X_train=train_x, y_train=train_y, X_test =test_x,y_test=test_y)
        best_model = my_fit(estimateur=estimateur,
                                        nb_iter=nb_iter, my_nb_splits=10,
                                       my_test_size=1 - taille_training,
                                        my_random_state=0,
                                        train_x=train_x, train_y=train_y,allure=allure,featurecolums=feature_columns)



        Plot_predictor_importance(best_model, feature_columns)



        # sauvegarde du modele
        save_mymodel(clf=best_model,allure=allure)

        # EVALUATION
        evaluation(mybest_model=best_model, train_x=train_x, test_x=test_x,
                           train_y=train_y, test_y=test_y, mode_debug=mode_debug)

    else:
        # CHARGEMENT DU MODELE
        best_model = load_mymodel(allure)




    return best_model, feature_columns, response_column


In [37]:
def jouerlescourse(nb_itera, allure, mode_debug=0, actualise=0):
    
    global best_model_1,   best_model_2, best_model_3, best_model_4, best_model_5,\
            feature_columns1, feature_columns2, feature_columns3,\
    feature_columns4,      feature_columns5,response_column
    
    #ENTRAINE_ALLURE
    best_model, feature_columns, response_column = entraine_allure(allure=allure,nb_iter=nb_itera,mode_debug=mode_debug,actualise=actualise)
    
    
    
    
    if allure==1:
        best_model_1 = best_model
        feature_columns1 = feature_columns
        
    if allure==2:
        best_model_2 = best_model
        feature_columns2 = feature_columns
        
    if allure==3:
        best_model_3 = best_model
        feature_columns3 = feature_columns
        
    if allure==4:
        best_model_4 = best_model
        feature_columns4 = feature_columns
        
    if allure==5:
        best_model_5 = best_model
        feature_columns5 = feature_columns
        
            
    jouer.construireFichierCSV(allure=allure , best_model=best_model, 
                                      feature_columns=feature_columns,  
                                        response_column=response_column,
                                       mode_debug=mode_debug)

In [ ]:
nb=1# NB ITERATION=actua)

mode_debug=0 # MODE DEBUG
actua=0# recalcul du model 0 = actua   1=pas actuia


jouerlescourse(nb_itera=nb, allure=1, mode_debug=mode_debug,actualise=actua)
jouerlescourse(nb_itera=nb, allure=2,mode_debug=mode_debug,actualise=actua)
jouerlescourse(nb_itera=nb, allure=3, mode_debug=mode_debug,actualise=actua)
jouerlescourse(nb_itera=nb, allure=4, mode_debug=mode_debug,actualise=actua)
jouerlescourse(nb_itera=nb, allure=5, mode_debug=mode_debug,actualise=actua)


---------------------ALLURE(  1  )------------------------------------------------
